In [1]:
stock = 'SAP'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    116.400002
2018-07-23    116.809998
2018-07-24    117.379997
2018-07-25    119.269997
2018-07-26    119.699997
2018-07-27    119.050003
2018-07-30    116.769997
2018-07-31    116.040001
2018-08-01    115.949997
2018-08-02    115.570000
2018-08-03    115.709999
2018-08-06    115.239998
2018-08-07    115.589996
2018-08-08    116.019997
2018-08-09    115.690002
2018-08-10    112.860001
2018-08-13    114.099998
2018-08-14    114.169998
2018-08-15    113.220001
2018-08-16    116.080002
2018-08-17    116.900002
2018-08-20    117.699997
2018-08-21    117.980003
2018-08-22    119.080002
2018-08-23    119.290001
2018-08-24    120.900002
2018-08-27    122.209999
2018-08-28    122.029999
2018-08-29    122.050003
2018-08-30    121.230003
                 ...    
2018-09-17    119.790001
2018-09-18    120.279999
2018-09-19    119.489998
2018-09-20    120.129997
2018-09-21    121.639999
2018-09-24    122.040001
2018-09-25    126.610001
2018-09-26    126.169998
2018-09-27    125.36

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


114.88428251828546
[119.51057429 120.97173681 109.61718266 105.78356401 118.53835481]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")